In [19]:
import pandas as pd
import bz2
import os
import json
import twitter_functions
import multiprocessing as mp
from time import time
import numpy as np
import sys

In [20]:
path_to_timeline='../data/timelines/API/IDF100_small/'
bzs = sorted([x for x in os.listdir(path_to_timeline) if x[-4:]=='json'])

In [21]:
len(bzs)

53

In [22]:
def get_tweets(index_block,n_blocks):
    df=pd.DataFrame()
    for bz in bzs[n_blocks*index_block:n_blocks*(index_block+1)]:
        print(str(bz))
        with open(path_to_timeline+bz, 'r') as f:
            tweets_user = pd.DataFrame(json.load(f))
            df = pd.concat([df,tweets_user])
    return df

In [24]:
# # START POOL OF CPUS
# try:
#     n_cpu = int(os.getenv("SLURM_CPUS_PER_TASK"))
#     print('INFER # CORES FROM PBS VARIABLE')
# except:
#     n_cpu = mp.cpu_count()
#     print('USE DEFAULT # CORES')

n_cpu = mp.cpu_count()
print('USE DEFAULT # CORES')
pool  = mp.Pool(processes=n_cpu)
print("# PROCESSORS:", n_cpu, "\n")

print("Read Tweets ...")
start = time()

# COMPUTE LIST OF RESULTS
n_blocks= len(bzs)//n_cpu + len(bzs)%n_cpu
print(n_blocks)
results = [pool.apply_async(get_tweets, args=(index_block,n_blocks)) for index_block in range(n_cpu)]
print(results)
print('concated vraiment')
tweets  = pd.concat([results[index_block].get() for index_block in range(n_cpu)]).reset_index(drop=True)
print('done')
#print(tweets)

pool.close()    
print("DONE IN", round(time() - start), "SEC")

USE DEFAULT # CORES
# PROCESSORS: 16 

Read Tweets ...
8
timelines-0-0-2-100-ece87d5d-99ca-4ecb-8ea9-cda188fe9e2b.json
timelines-0-0-0-100-0aeb7cd3-9724-48ff-8718-36fdd6c99ecd.json
timelines-0-0-1-100-5866da3a-c37c-4d2f-b3ab-fcf656e19f82.json
timelines-0-0-4-100-69a7ce01-65f0-4316-9ca1-48c93ab5cf1f.json
timelines-0-0-5-100-44c02157-a57e-4b32-a3bf-0c02f448ae0d.json
timelines-0-0-6-100-3457a915-5e76-459b-85f7-b0b82f2b6f7c.json
timelines-0-0-7-100-addb9975-1abe-416a-918b-43d8bb90acb7.json
[<multiprocessing.pool.ApplyResult object at 0x7f9ec6bf8b50>, <multiprocessing.pool.ApplyResult object at 0x7f9e489c8310>, <multiprocessing.pool.ApplyResult object at 0x7f9e489c8410>, <multiprocessing.pool.ApplyResult object at 0x7f9e489c8490>, <multiprocessing.pool.ApplyResult object at 0x7f9e489c8590>, <multiprocessing.pool.ApplyResult object at 0x7f9e489c82d0>, <multiprocessing.pool.ApplyResult object at 0x7f9e489c8790>, <multiprocessing.pool.ApplyResult object at 0x7f9e489c8890>, <multiprocessing.poo

In [25]:
len(tweets)

10657619

In [26]:
len(tweets['user_id'].unique())

5298

In [27]:
tweets.to_json(r'../data/timelines_concat/tweets_historic_idf_100.json', orient='index')